<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/new_version_3_KARACartoonize_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepair

#Session settings

In [9]:
# Pick a filesystem
filesystem = "colab" #@param ["colab", "drive"]

In [10]:
# Pick a style
all_styles=["shinkai", "hayao", "hosoda", "paprika"]
styles = "hayao" #@param ["shinkai", "hayao", "hosoda", "paprika","all"]

In [11]:
# pick source 
source = 'upload' #@param ["upload","url"]

In [12]:
# path to video
vid_path =  '/content/GHOSTEMANE_AI.mp4'

# set filesystem

In [5]:
import os
os.chdir("/content")
if filesystem == 'drive':
  # sync google drive
  from google.colab import drive
  import os
  # if drive is needed uncomment
  print('Google drive ',end='')
  drive.mount('/content/drive',force_remount=True)
  print('Root folder set to ',end='')
  root = '/content/drive/My drive/Cartoonizer'
  print(root)
else:
  print('Root folder set to ',end='')
  root = '/content/Cartoonizer'
  print(root)

Root folder set to /content/Cartoonizer


# import needed stuff

In [ ]:
# imports
os.chdir('/content')
import time,os,glob,cv2, itertools
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
!apt install tree

# #pytorch implement pretrained files
# !git clone https://github.com/Yijunmaverick/CartoonGAN-Test-Pytorch-Torch.git
# os.chdir('CartoonGAN-Test-Pytorch-Torch')
# os.mkdir('uploaded_images')
# !bash pretrained_model/download_pth.sh
# os.chdir('/content')

#tensorflow implement
repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/{repo}.git
os.chdir(os.path.join(repo))
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install imageio
!pip install ffmpeg ffprobe
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install
  
# from subprocess import call
import tensorflow as tf
tf.__version__

os.chdir("/content")
# get split and recombine scripts
!git clone https://github.com/graphific/DeepDreamVideo.git
# clear the long output
clear_output()

## Functions

In [ ]:
#function defines
def resize(image,window_height = 1000):
    aspect_ratio = float(image.shape[1])/float(image.shape[0])
    window_width = window_height/aspect_ratio
    image = cv2.resize(image, (int(window_height),int(window_width)))
    return image

def download_image(url,input_file_name):
  !curl -o {input_file_name} {url}
  img = cv2.imread(input_file_name, cv2.IMREAD_UNCHANGED)
  return img

def numfiles(folder):#get all frames
    searchedfiles = glob.glob(folder)
    files = sorted( searchedfiles, key = lambda file: os.path.getctime(file))
    return len(files)

## Set I/O

In [ ]:
# set i/o
input_folder = 'input'
output_folder = 'output'
# check if paths are in place
if not os.path.exists(root):
  os.mkdir(root)
  os.mkdir(root+'/'+output_folder)  
  os.mkdir(root+'/'+input_folder)
  

## Split Audio from video and split video in frames


In [ ]:
os.chdir("/content")

In [ ]:
#!rm -r /content/DeepDreamVideo
#!rm -r /content/Cartoonizer
#!rm -r /content/CartoonGan-tensorflow

In [ ]:
os.chdir("DeepDreamVideo")
if os.path.exists("process_folder") == False:
    os.makedirs("process_folder")
if os.path.exists("Out_process_folder") == False:
    os.makedirs("Out_process_folder")
if os.path.exists("/content/MusicVideos") == False:
    os.makedirs("/content/MusicVideos")

In [ ]:
vid_to_split = vid_path

In [ ]:
os.chdir('/content/DeepDreamVideo')

In [ ]:
import subprocess
def RunSubProcess(cmd):
    result = subprocess.run([cmd], shell=True, capture_output=True, text=True)
    print(result.stdout)
    return result

In [ ]:
RunSubProcess('sh /content/DeepDreamVideo/1_movie2frames.sh GodSmack ' + vid_to_split + ' process_folder jpg')

In [ ]:
print(numfiles('/content/DeepDreamVideo/process_folder/*.jpg'))

In [20]:

os.chdir("/content/CartoonGan-tensorflow")

In [21]:
if source == 'url':
  # Set the img url
  img_url = input('Enter the image url : ')
  # extract the name and add the style
  img_name = img_url.split('.')
  img_ext = img_name[len(img_name)-1]
  img_name = img_name[len(img_name)-2]
  img_name = img_name.split('/')[2]
  img_name = img_name + '_' + styles +'.'+ img_ext
  print(img_in_folder+img_name)
  img_array = download_image(img_url,img_in_folder+img_name)

In [22]:
# Set i/o folders
#img_in_folder = '/content/drive/MyDrive/Colab Notebooks/datasets/garbage/llery_2k14s1'
#img_in_folder =  root+'/'+input_folder
img_out_folder = root+'/'+output_folder

In [23]:
RunSubProcess('ffprobe ' + vid_to_split)

CompletedProcess(args=['ffprobe /content/GHOSTEMANE_AI.mp4'], returncode=0, stdout='', stderr="ffprobe version 4.2.7-0ubuntu0.1 Copyright (c) 2007-2022 the FFmpeg developers\n  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex

In [24]:

# Import everything needed to edit video clips
from moviepy.editor import *
import moviepy.editor as mp
import os

In [25]:
movie_in = vid_to_split

split = movie_in.replace('.mp4','').split('/')
video_name = split[ len(split) - 1 ]

In [26]:
process_folder = '/content/' + video_name
img_in = process_folder + '/frames'
#img_out = process_folder +'/processed'
img_out_new = "/content/DeepDreamVideo/process_Out_folder"
#img_rez = process_folder +'/rez'
img_rez_new = "/content/DeepDreamVideo/process_folder"



In [ ]:
folderList = [process_folder,img_in,img_out_new,img_rez_new]
os.chdir('/content/DeepDreamVideo')
for folder in folderList:
    if os.path.exists(folder):
        RunSubProcess('rm -r ' + folder+'/*' )
    else:
        os.makedirs(folder)
#!mkdir {img_in}
#!mkdir {img_out}
#!mkdir {img_rez}

#!mkdir {img_out_new}
#!mkdir {img_rez_new}



## Cartoon time

In [ ]:
import subprocess
import time, sys
import threading
def oneCall():
    subprocess.call(['python','one.py'])

def twoCall():
    subprocess.call(['python','two.py'])


class progress_bar_loading(threading.Thread):

    def run(self):
            global stop
            global kill
            print('Loading....  ')
            sys.stdout.flush()
            i = 0
            while stop != True:
                    if (i%4) == 0: 
                        sys.stdout.write('\b/')
                    elif (i%4) == 1: 
                        sys.stdout.write('\b-')
                    elif (i%4) == 2: 
                        sys.stdout.write('\b\\')
                    elif (i%4) == 3: 
                        sys.stdout.write('\b|')

                    sys.stdout.flush()
                    time.sleep(0.2)
                    i+=1

            if kill == True: 
                print('\b\b\b\b ABORT!',
            else: 
                print('\b\b\b done!',


kill = False      
stop = False
p = progress_bar_loading()
p.start()

try:
    #anything you want to run.
    oneCall()
    twoCall()
    time.sleep(1)
    stop = True
except KeyboardInterrupt or EOFError:
         kill = True
         stop = True

In [31]:
def RunSub():
  
  RunSubProcess('python cartoonize.py \
    --skip_comparison \
    --keep_original_size \
    --batch_size 10 \
    --styles ' + styles + ' \
    --input_dir ' + img_rez_new + ' \
    --output_dir ' + img_out_new)
  
def Run():
  
  !python cartoonize.py \
    --skip_comparison \
    --keep_original_size \
    --batch_size 10 \
    --styles {styles} \
    --input_dir {img_rez_new} \
    --output_dir {img_out_new}

  
if styles == 'all':
    print('Looping over all styles')
    for styles in ["shinkai", "hayao", "hosoda", "paprika"]:
        Run()    
else:
    print('Running with '+styles+" style")
    Run()



Running with hayao style
2023-05-11 21:50:53.774533: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-05-11 21:50:55] [Cartoonizer] [INFO] Transformed images will be saved to `/content/DeepDreamVideo/process_Out_folder` folder.
2023-05-11 21:50:56.263811: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
[2023-05-11 21:50:57] [Cartoonizer] [INFO] Cartoonizing images using hayao style...
[2023-05-11 21:50:57] [Cartoonizer] [INFO] Preparing to transform 4227 images from `/content/DeepDreamVideo/process_folder` directory...
1/1 [==============================] - 0s 338ms/step
Transforming: 100% 4227/4227 [17:38<00:00,  3.99it/s, File=00000272.jpg]
[2023-05-11 22:08:35] [Cartoonizer] [INFO] Total processing time: 0:17:40.719992


In [ ]:
  #!python cartoonize.py \
  # --comparison_view horizontal \
  #  --keep_original_size \
  #  --batch_size 1 \
  #  --styles '{styles}' \
  #  --input_dir '{img_in_resized}' \
  #  --output_dir '{img_in_processed}' 

  #!mv '{img_in_processed}/comparison' '{img_in_processed}/{styles}/comparison' 
  


##Dream time


# method defining

In [ ]:
# Download an image and read it into a NumPy array.
def ImageFromPath(path, max_dim=None):
  img = PIL.Image.open(path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)


# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
  name = url.split('/')[-1]
  image_path = tf.keras.utils.get_file(name, origin=url)
  img = PIL.Image.open(image_path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Normalize an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Display an image
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))

def calc_loss(img, model):
  # Pass forward the image through the model to retrieve the activations.
  # Converts the image into a batch of size 1.
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  return  tf.reduce_sum(losses)


class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
  )
  def __call__(self, img, steps, step_size):
      print("Tracing")
      loss = tf.constant(0.0)
      for n in tf.range(steps):
        with tf.GradientTape() as tape:
          # This needs gradients relative to `img`
          # `GradientTape` only watches `tf.Variable`s by default
          tape.watch(img)
          loss = calc_loss(img, self.model)

        # Calculate the gradient of the loss with respect to the pixels of the input image.
        gradients = tape.gradient(loss, img)

        # Normalize the gradients.
        gradients /= tf.math.reduce_std(gradients) + 1e-8 
        
        # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
        # You can update the image by directly adding the gradients (because they're the same shape!)
        img = img + gradients*step_size
        img = tf.clip_by_value(img, -1, 1)

      return loss, img

def run_deep_dream_simple(img, steps=100, step_size=0.01):
  # Convert from uint8 to the range expected by the model.
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  img = tf.convert_to_tensor(img)
  step_size = tf.convert_to_tensor(step_size)
  steps_remaining = steps
  step = 0
  while steps_remaining:
    if steps_remaining>100:
      run_steps = tf.constant(100)
    else:
      run_steps = tf.constant(steps_remaining)
    steps_remaining -= run_steps
    step += run_steps

    loss, img = deepdream(img, run_steps, tf.constant(step_size))
    
    display.clear_output(wait=True)
    show(deprocess(img))
    print ("Step {}, loss {}".format(step, loss))


  result = deprocess(img)
  display.clear_output(wait=True)
  show(result)

  return result

In [ ]:


image_path = '/content/DeepDreamVideo/process_folder/00001119.jpg'
# Downsizing the image makes it easier to work with.
original_img = ImageFromPath(image_path, max_dim=1000)
#show(original_img)
#display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
# Maximize the activations of these layers
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]
print(layers)
# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)
deepdream = DeepDream(dream_model)


In [ ]:
# run simple 100 steps
dream_img = run_deep_dream_simple(img=original_img, 
                                  steps=100, step_size=0.01)

In [ ]:
import time
start = time.time()
OCTAVE_SCALE = 1.30
img = tf.constant(np.array(original_img))
base_shape = tf.shape(img)[:-1]
float_base_shape = tf.cast(base_shape, tf.float32)

for n in range(-2, 3):
  new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)
  img = tf.image.resize(img, new_shape).numpy()
  img = run_deep_dream_simple(img=img, steps=50, step_size=0.01)

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
show(img)
end = time.time()
end-start

In [ ]:
def random_roll(img, maxroll):
  # Randomly shift the image to avoid tiled boundaries.
  shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
  img_rolled = tf.roll(img, shift=shift, axis=[0,1])
  return shift, img_rolled

In [ ]:
shift, img_rolled = random_roll(np.array(original_img), 512)
show(img_rolled)

In [ ]:
class TiledGradients(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[2], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.int32),)
  )
  def __call__(self, img, img_size, tile_size=512):
    shift, img_rolled = random_roll(img, tile_size)

    # Initialize the image gradients to zero.
    gradients = tf.zeros_like(img_rolled)
    
    # Skip the last tile, unless there's only one tile.
    xs = tf.range(0, img_size[1], tile_size)[:-1]
    if not tf.cast(len(xs), bool):
      xs = tf.constant([0])
    ys = tf.range(0, img_size[0], tile_size)[:-1]
    if not tf.cast(len(ys), bool):
      ys = tf.constant([0])

    for x in xs:
      for y in ys:
        # Calculate the gradients for this tile.
        with tf.GradientTape() as tape:
          # This needs gradients relative to `img_rolled`.
          # `GradientTape` only watches `tf.Variable`s by default.
          tape.watch(img_rolled)

          # Extract a tile out of the image.
          img_tile = img_rolled[y:y+tile_size, x:x+tile_size]
          loss = calc_loss(img_tile, self.model)

        # Update the image gradients for this tile.
        gradients = gradients + tape.gradient(loss, img_rolled)

    # Undo the random shift applied to the image and its gradients.
    gradients = tf.roll(gradients, shift=-shift, axis=[0,1])

    # Normalize the gradients.
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    return gradients 

In [ ]:
get_tiled_gradients = TiledGradients(dream_model)

In [ ]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
  base_shape = tf.shape(img)
  img = tf.keras.utils.img_to_array(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  initial_shape = img.shape[:-1]
  img = tf.image.resize(img, initial_shape)
  for octave in octaves:
    # Scale the image based on the octave
    new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
    new_size = tf.cast(new_size, tf.int32)
    img = tf.image.resize(img, new_size)

    for step in range(steps_per_octave):
      gradients = get_tiled_gradients(img, new_size)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1)

      if step % 10 == 0:
        display.clear_output(wait=True)
        show(deprocess(img))
        print ("Octave {}, Step {}".format(octave, step))
    
  result = deprocess(img)
  return result

In [ ]:
import cv2
img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)

display.clear_output(wait=True)
img = tf.image.resize(img, base_shape)
img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
#show(img)

plt.axis("off")
plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
plt.show()
cv2.imwrite('colortrip.png',img_grayscale)

In [ ]:
!rm -r /content/CartoonGan-tensorflow/deep_dream

In [ ]:
!git clone https://github.com/gsurma/deep_dream.git

In [ ]:
!cd deep_dream && python /content/CartoonGan-tensorflow/deep_dream/deep_dream.py /content/DeepDreamVideo/process_folder/00000013.jpg

In [ ]:
from io import StringIO

In [ ]:
!python /content/PyDeepDream/deepdream.py --help

In [ ]:
##Back to video


In [32]:
!cd /content/DeepDreamVideo

In [35]:
#TEST 
!sh /content/DeepDreamVideo/3_frames2movie.sh ffmpeg /content/DeepDreamVideo/process_Out_folder/hayao /content/GHOSTEMANE_AI.mp4 jpg


/content/DeepDreamVideo/3_frames2movie.sh: 8: [: png: unexpected operator
/content/DeepDreamVideo/3_frames2movie.sh: 25: [: avconv: unexpected operator
/content/DeepDreamVideo/3_frames2movie.sh: 38: [: mplayer: unexpected operator
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libo

In [ ]:
#Carto0nized frames 2 video
!sh 3_frames2movie.sh ffmpeg /content/DeepDreamVideo/Out_process_folder/paprika /content/videoplayback.mp4 jpg

In [ ]:
#Compair carto0nized sidebyside with original 2 video
!sh 3_frames2movie.sh ffmpeg /content/DeepDreamVideo/Out_process_folder/comparison /content/videoplayback.mp4 jpg

Train own cartoonstyle

Prepare Dataset
You also need to prepare your own dataset and arrange the images under datasets folder as below:

datasets
└── YourDataset [your dataset name]
    ├── testA [(must) 8 real-world images for evaluation]
    ├── trainA [(must) (source) real-world images]
    ├── trainB [(must) (target) cartoon images]
    └── trainB_smooth [(must, but can be generated by running scripts/smooth.py) cartoon images with smooth edges]
trainA and testA folders contain real-world images, while trainB contain images with desired cartoon style. Notice that 8 images in testA folder will be evaluated after each epoch, so they should not appear in trainA.

In order to generate trainB_smooth, you can run scripts/smooth.py:

python path/to/smooth.py --path path/to/datasets/YourDataset  # YourDataset should contain trainB for executing this script
smooth.py credit to taki0112 https://github.com/taki0112/CartoonGAN-Tensorflow/blob/master/edge_smooth.py

Start training
Although you may have to tune hyperparameters to generate best result for your own datasets, train following settings that we found effective can be your starting point.

If you get more than 16GB memory in your GPU, you can try these settings (Note that --light indicates that we are training GAN with a light-weight generator):


In [ ]:

python train.py \
    --batch_size 8 \
    --pretrain_epochs 1 \
    --content_lambda .4 \
    --pretrain_learning_rate 2e-4 \
    --g_adv_lambda 8. \
    --generator_lr 8e-5 \
    --discriminator_lr 3e-5 \
    --style_lambda 25. \
    --light \
    --dataset_name {your dataset name}

# Note that style_lambda is for style loss (source). If your GPU does not have 16GB memory, you can use a smaller batch_size and use lower learning rates accordingly. For example, for batch_size = 4, you can try:

python train.py \
    --batch_size 4 \
    --pretrain_epochs 1 \
    --content_lambda .4 \
    --pretrain_learning_rate 1e-4 \
    --g_adv_lambda 8. \
    --generator_lr 4e-5 \
    --discriminator_lr 1.5e-5 \
    --style_lambda 25. \
    --light \
    --dataset_name {your dataset name}

#train-demo
#Detailed log messages, model architecture and progress bar are all provided. This enable you to gain a better understanding of what is happening when training a CartoonGAN.

#Choose model architecture
#Notice that we specified --light in our previous example:

python train.py \
    ...
    --light \
    ...
#When specified, train.py will initialize a light-weight generator for training a CartoonGAN.

#When we design the light-weight generator, ShuffleNet V2 is taken as our reference. This generator is designed to minimalize inference time while achieving similar effect. We will make some minor adjustments to discriminator as well when --light is specified.

#generator

#Generator proposed by the original CartoonGAN authors

#To train a CartoonGAN with the original generator/discriminator architecture proposed by the CartoonGAN authors, simply remove --light option:

python train.py \
    --batch_size 8 \
    --pretrain_epochs 1 \
    --content_lambda .4 \
    --pretrain_learning_rate 2e-4 \
    --g_adv_lambda 8. \
    --generator_lr 8e-5 \
    --discriminator_lr 3e-5 \
    --style_lambda 25. \
    --dataset_name {your dataset name}


#Monitor your training progress
#In our repo, TensorBoard is integrated perfectly so you can monitor model's performance easily by:

#tensorboard --logdir runs
#After training for awhile, you should be able to see something like this:

#tensorboard-metrics

#In addition to metrics and loss functions, it is good practice to keep an eye on the images generated by GAN during training as well. Using our script, monitoring generated images on TensorBoard is a no-brainer:

#tensorboard-image-demo

#For further details about training, we recommend reading train.py.


##Check results

In [ ]:

#Inference with trained checkpoint
#Once your generator is well-trained, you can try cartoonizing with your trained checkpoint:

# You should specify --light if your model is trained with --light
# If you didn't specify --light on your training, you should remove --light
# default of --out_dir is out

python inference_with_ckpt.py \
    --m_path path/to/model/folder \
    --img_path path/to/your/img.jpg \
    --out_dir path/to/your/desired/output/folder \
    --light

#And generated image will be saved to path/to/your/desired/output/folder/img.jpg.

#Export checkpoint to SavedModel and tfjs
#Once your generator is well-trained, you can export your model to tfjs model and SavedModel:

# You should specify --light if your model is trained with --light
# If you didn't specify --light on your training, you should remove --light
# default of --out_dir is exported_models


python export.py \
    --m_path path/to/model/folder \
    --out_dir path/to/your/desired/export/folder \
    --light


#And exported tfjs model and SavedModel will be saved to path/to/your/desired/export/folder.

#Note that the whole model architecture is saved to SavedModel and tfjs model, so you don't need to specify --light anymore.

#You can try cartoonizing with your exported SavedModel:

# default of --out_dir is out

python inference_with_saved_model.py \
    --m_path path/to/your/exported/SavedModelFolder \
    --img_path path/to/your/img.jpg \
    --out_dir path/to/your/desired/output/folder


#And generated image will be saved to path/to/your/desired/output/folder/img.jpg.

#We trained 2 model checkpoints and put them in the repo: light_paprika_ckpt and light_shinkai_ckpt (~7MB Total). You can play around the ckpts with inference_with_ckpt.py and export.py.

#Also, we put our exported shinkai and paprika SavedModels in exported_models (~11MB Total). You can play around the SavedModels with inference_with_saved_model.py.

#Image generated using our exported_models/light_paprika_SavedModel (left: original; right: generated): origami_demo.jpg